# Exercise 1 - 3D - 2D

In [82]:
import numpy as np
import cv2 as cv2
from numpy.linalg import inv, pinv, norm
import matplotlib.pyplot as plt

Recall from the slides the steps from Algorithm 3:

![title](algorithm_3.png)

![title](PnP.png)

# Exercise 1a)
The steps 1)-2.1) has already been done, and is saved in corresponding files. The exercise is to implement step 2.2) by filling in the missing code below

In [119]:
def featureTracking(prev_img, next_img, prev_points, world_points):
    """
    Use OpenCV to find the prev_points from the prev_img in the next_img
    Remember to remove points that could not be found from prev_points, next_points, and world_points
    hint: status == 1
    """
    prev_points_list = []
    world_points_list = []
    next_points_list = []
    
    feature_params = dict(maxCorners=100,
              qualityLevel=0.3, 
              minDistance=7)
    
    lk_params = dict(winSize=(21, 21),
                 maxLevel=3,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
    
    #prev_img_gray = cv2.cvtColor(prev_img, cv2.COLOR_RGB2GRAY)
    #next_img_gary = cv2.cvtColor(next_img, cv2.COLOR_RGB2GRAY)
    #feat1 = cv2.goodFeaturesToTrack(prev_img_gray, **feature_params)
    #featt = np.int0(feat1)
    
    #next_points, status, _ = ...
    next_points, status, error = cv2.calcOpticalFlowPyrLK(prev_img, next_img, prev_points, None, **lk_params)
    
    #next_points = next_points[status==1]
    #prev_points = prev_points[status==1]
    #world_points = world_points[status==1]
    for i in range(len(prev_points)):
        if(status[i] == 1):
            prev_points_list.append(prev_points[i])
            world_points_list.append(world_points[i])
            next_points_list.append(next_points[i])
    
    world_points = np.array(world_points_list, dtype = np.float64)
    prev_points = np.array(prev_points_list, dtype = np.float64)
    next_points = np.array(next_points_list, dtype = np.float64) 
    
    return world_points, prev_points, next_points

Hint: Exercise 4 in week 2

# Exercise 1b)
Continue the algorithm by implementing step 2.3)

In [120]:
K = np.array([[7.188560e+02, 0.000000e+00, 6.071928e+02], # camera matrix
              [0, 7.188560e+02, 1.852157e+02],
              [0, 0, 1]])

reference_img = np.load("img_" + str(0) + ".npy")

for t in range(1, 6):

    # the image at current time=t
    curImage = np.load("img_" + str(t) + ".npy")
    # the 3D landmarks in the world coordinates which have been computed in time=t-1
    landmark_3D = np.load("landmark_" + str(t-1) + ".npy")
    # the 2D coordinates of the 3D points in the previous frame at time=t-1
    reference_2D = np.load("reference_2D_" + str(t-1) + ".npy")
    
    # the 2D landmarks at the current time = t
    landmark_3D, reference_2D, tracked_2Dpoints = featureTracking(reference_img, 
                                                                  curImage, 
                                                                  reference_2D,
                                                                  landmark_3D)
    """
    Using OpenCV, implement PnP using Ransac
    """
    #_, rvec, tvec, inliers = ...
    #bring points from model coordinate system to camera system  
    ret, rvec, tvec, inliers = cv2.solvePnPRansac(landmark_3D, tracked_2Dpoints, K, None)
    """
    Transform the translation and rotation into the world frame
    """
    
    tvec = pinv(tvec)
    tvec = tvec.reshape((3,1))
    rvec = pinv(rvec)
    rvec = rvec.reshape((3,1))
    print("cycle: %d\n "%t)
    print(tvec[0], tvec[1], tvec[2], rvec[0], rvec[1], rvec[2])

    # update for next timestep
    reference_img = curImage


cycle: 1
 
[483.41260207] [294.51377037] [343.46146785] [-3597.05673022] [-3572.9968705] [4785.31137953]
cycle: 2
 
[0.00318923] [0.01593734] [-1.47947453] [-101.80422063] [153.111999] [-114.80733333]
cycle: 3
 
[0.00032921] [0.00596972] [-0.72590177] [-51.5248016] [106.19795174] [-14.08760801]
cycle: 4
 
[0.00182074] [0.00337485] [-0.47615848] [-33.42294736] [73.56745955] [-5.4424258]
cycle: 5
 
[0.00049759] [0.00241971] [-0.3528914] [-19.22820564] [55.25493405] [1.43781174]


Hint: The output should look similar to:

[-0.00110282] [-0.00067164] [-0.00078343] [-7.40069212e-05] [-7.35119065e-05] [9.84544279e-05]

[-0.00363946] [-0.00875075] [0.67580842] [-0.0021666] [0.00325853] [-0.00244333]

[-0.01096271] [-0.01635663] [1.3774094] [-0.00364615] [0.0075151] [-0.00099691]

[-0.0315663] [-0.02560111] [2.0996797] [-0.00509583] [0.01121646] [-0.00082978]

[-0.04971858] [-0.03532535] [2.8330071] [-0.00561424] [0.0161333] [0.00041981]

# Exercise 1c)
What approximate direction did the camera move in?